In [1]:
# Import libraries adn load dependencies
import torch
import numpy as np
import pandas as pd

# Models:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification # for german
from pattern.en import sentiment # for english
import nltk # for english
from nltk.sentiment import SentimentIntensityAnalyzer # for english
nltk.download('vader_lexicon') # for english
from pysentimiento import create_analyzer # for spanish

# Accuracy
from utils import sentiment_translated_scores, evaluate_performance

C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\joana\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# More sentiment models and their accuracy per language

## Intro
TODO

full text
extracted sentence

full condensed
extracted sentence

### Load labeled data: d1.1
Explanation why we used d1.1 and that is already with extracted sentence 

In [43]:
# Load labeled CSV files into a DataFrame
df_de_1 = pd.read_csv('https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/Preprocessing/data_clean/labeled-data/labeled-de_clean_1-1.csv', sep=';')
df_en_1 = pd.read_csv('https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/Preprocessing/data_clean/labeled-data/labeled-en_clean_1-1_not101010.csv')
df_es_1 = pd.read_csv('https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/Preprocessing/data_clean/labeled-data/labeled-es_clean_1-1.csv', sep=';')

### Load labeled data: data condensed for english
Explanation why we used data condensed for english

In [15]:
# Load labeled CSV file into a DataFrame
df_en_con = pd.read_csv('https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/Preprocessing/data_clean/labeled-data/labeled-en_clean_con_sen.csv')

## German
For the german data we used one additional model to the multilingual only, because then we focused on clustering.

### Model: AutoTokenizer and AutoModelForSequenceClassification.from_pretrained("oliverguhr/german-sentiment-bert")
The model used was trained on 1.834 million German-language samples sepcifically for sentiment classification.
https://huggingface.co/oliverguhr/german-sentiment-bert

In [25]:
def german_sentiment_model(df):
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("oliverguhr/german-sentiment-bert")
    model = AutoModelForSequenceClassification.from_pretrained("oliverguhr/german-sentiment-bert")

    # Create an empty list to store the sentiment scores
    sentiment_scores = []

    # Iterate over the 'data' column in the DataFrame
    for text in df['data']:
        # Tokenize the input text
        tokens = tokenizer.encode_plus(text, padding="max_length", truncation=True, max_length=128,
                                       return_tensors="pt")

        # Perform the sentiment analysis
        with torch.no_grad():
            logits = model(**tokens)[0]

        # Convert logits to predicted label (positive/negative)
        predicted_label = torch.argmax(logits, dim=1).item()
        sentiment = "positive" if predicted_label == 1 else "negativ"

        # Append the sentiment score to the list
        sentiment_scores.append(sentiment)

    # Add the sentiment scores as a new column in the DataFrame
    df['sentiment_bert'] = sentiment_scores

    return df


# Perform sentiment analysis on the DataFrame
df_de_1 = german_sentiment_model(df_de_1)

# Print the updated DataFrame
print(df_de_1)



#ACTION: NOTE FOR JOANA: check whether this makes a new dataframe instead of appending a column. If so: Change & check what 

                                                  data    player language  \
14   zweimal verwandelte palacios, es wurde ein ube...  palacios       de   
23   schalke: reis nimmt den spieler ausdrucklich i...  palacios       de   
31   palacios (r.) verletzte sich leicht im hinspie...  palacios       de   
41   "auf jeden fall" sei das ein ganz grosser tag,...  palacios       de   
58   beim abschlusstraining trug letzterer zumindes...  palacios       de   
63   bayerleverkusen argentinischer weltmeister pal...  palacios       de   
83   "es gibt im moment nicht viel besseres in der ...  frimpong       de   
93   der fc bayern munchen interessiert sich nach i...  frimpong       de   
95   frimpong hat noch bis vertrag in bayerleverkus...  frimpong       de   
102  hincapie (l.) und tah bedanken sich fur die fa...  frimpong       de   
120  diaby, wirtz, adam hlozek und frimpong teilten...  frimpong       de   
123     bayerleverkusen torschutze amiri (rechts) u...  frimpong       de   

In [17]:
#ACTION: Show distribution? If so: Do for all

#### Evaluate model performance for german bert model
TODO and todo code

In [26]:
# Drop rows where 'Label' is NaN or empty
df_de_1.dropna(subset=['Label'], inplace=True)

# Print the updated DataFrame
df_de_1_pos_neg = df_de_1[df_de_1['Label'] != 'neutral']

In [41]:
print('Performance evaluation for oliverguhr/german-sentiment-bert')
accuracy_de, unique_predicted_de, confusion_matrix_de, classification_report_de = evaluate_performance(df_de_1_pos_neg, 'sentiment_bert', 'Label')

print('Accuracy: ', accuracy_de,' Unique labels: ', unique_predicted_de)

print('Confusion matrix: ')
print(confusion_matrix_de)
print('Classification report: ')
print(classification_report_de)

Performance evaluation for oliverguhr/german-sentiment-bert
Accuracy:  0.5  Unique labels:  ['negativ']
Confusion matrix: 
         negativ  positiv
negativ       10        0
positiv       10        0
Classification report: 
              precision    recall  f1-score   support

     negativ       0.50      1.00      0.67        10
     positiv       0.00      0.00      0.00        10

    accuracy                           0.50        20
   macro avg       0.25      0.50      0.33        20
weighted avg       0.25      0.50      0.33        20



C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\joana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344

## English - d1.1
TODO

### Why we used the pipeline for the english and spanish models
TODO


### Model 1: sentiment-analysis from bert-base-uncased
TODO
https://huggingface.co/bert-base-uncased

In [45]:
# Initiate model
sentiment_classifier_en = pipeline('sentiment-analysis', model='bert-base-uncased')

# Perform model and translate scores into two- and threedimensional labels
sentiment_2_scores, sentiment_2_labels, sentiment_3_labels = sentiment_translated_scores(df_en_1, sentiment_classifier_en)

# Create columns and assign score and labels to the columns
df_en_1['sentiment_2_score_bert'] = sentiment_2_scores
df_en_1['sentiment_2_label_bert'] = sentiment_2_labels
df_en_1['sentiment_3_label_bert'] = sentiment_3_labels

In [51]:
# Model imported as sentiment
def sentiment_translate_scores(df, sentiment_score):
    # Iterate over the score column in the DataFrame
    for score in df[sentiment_score]:        
        # Convert sentiment score to two-dimensional label (positive/negative)
        sentiment_2_label = "positiv" if score > 0 else "negativ"
        sentiment_2_labels.append(sentiment_2_label)

        # Convert sentiment score to three-dimensional label (positive/neutral/negative)
        if sentiment_score > 0.2:
            sentiment_3_label = "positiv"
        elif sentiment_score < -0.2:
            sentiment_3_label = "negativ"
        else:
            sentiment_3_label = "neutral"
        sentiment_3_labels.append(sentiment_3_label)

    return sentiment_2_labels, sentiment_3_labels

# Perform model and translate scores into two- and threedimensional labels
sentiment_2_scores, sentiment_2_labels, sentiment_3_labels = sentiment_translate_scores(df_en_1, 'sentiment_2_score_bert')

df_en_1

TypeError: '>' not supported between instances of 'str' and 'float'

### Model 2: sentiment form pattern.en
TODO

https://stackabuse.com/python-for-nlp-introduction-to-the-pattern-library/

In [23]:
# Function to get sentiment polarity
def get_sentiment(text):
    sentiment_score = sentiment(text)[0]
    return sentiment_score

# Apply sentiment analysis to the "data" column and store the sentiment in a new column "sentiment_pattern"
df_en_1['sentiment_pattern'] = df_en_1['data'].apply(sentiment)

# Print the updated dataframe
df_en_1.head()

,data,player,language,publishedAt,Label,sentiment_bert,sentiment_pattern
0,ten if you included the toe-poked volley to te...,palacios,en,2023-02-16T23:56:00Z,NaN,0.640392,0.000000
1,bayerleverkusen took the lead again in the st ...,palacios,en,2023-02-23T20:50:50Z,NaN,0.705472,0.208333
2,wissam ben yedder levelled straight away from ...,palacios,en,2023-02-23T20:53:59Z,positiv,0.702478,0.100000
3,"midfielders: leandro paredes (juventus), angel...",palacios,en,2023-03-03T16:40:46Z,neutral,0.757339,0.000000
4,midfielders: rodrigo de paul (atletico madrid)...,palacios,en,2023-03-03T18:17:37Z,neutral,0.754591,0.200000


### Model 3: Sentiment Intensity Analyzer from nltk

In [28]:
# Create an instance of the VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Function to get sentiment polarity
def get_sentiment(text):
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores['compound']


# Apply sentiment analysis to the "data" column and store the sentiment in a new column "sentiment_nltk"
df_en_1['sentiment_nltk'] = df_en_1['data'].apply(get_sentiment)

# Print the updated dataframe
df_en_1.head()


,data,player,language,publishedAt,Label,sentiment_bert,sentiment_pattern,sentiment_nltk
0,ten if you included the toe-poked volley to te...,palacios,en,2023-02-16T23:56:00Z,NaN,0.640392,0.000000,0.0000
1,bayerleverkusen took the lead again in the st ...,palacios,en,2023-02-23T20:50:50Z,NaN,0.705472,0.208333,-0.0516
2,wissam ben yedder levelled straight away from ...,palacios,en,2023-02-23T20:53:59Z,positiv,0.702478,0.100000,0.2263
3,"midfielders: leandro paredes (juventus), angel...",palacios,en,2023-03-03T16:40:46Z,neutral,0.757339,0.000000,0.0000
4,midfielders: rodrigo de paul (atletico madrid)...,palacios,en,2023-03-03T18:17:37Z,neutral,0.754591,0.200000,0.0000


#### Evaluate model performance for all english models
TODO 

In [30]:
# Drop rows where 'Label' is NaN or empty
df_en_1.dropna(subset=['Label'], inplace=True)

# Print the updated DataFrame
df_en_1_pos_neg = df_en_1[df_en_1['Label'] != 'neutral']

In [31]:
print('Performance evaluation for bert-base-uncased')

accuracy_en_bert, unique_predicted_en_bert, confusion_matrix_en_bert, classification_report_en_bert = evaluate_performance(df_en_1_pos_neg, 'sentiment_bert', 'Label')

print('Accuracy: ', accuracy_en_bert,' Unique labels: ', unique_predicted_en_bert)

print('Confusion matrix: ')
print(confusion_matrix_en_bert)
print('Classification report: ')
print(classification_report_en_bert)

Performance evaluation for bert-base-uncased
Accuracy for bert: 0.00%
Unique labels for bert:
[0.70247829 0.71411204 0.65856969 0.70102882 0.71734881 0.70871013
 0.60306615 0.7403965  0.71762341 0.68655038 0.68715137 0.69438171
 0.72599542 0.6514957  0.6753307  0.71295387 0.64372659]


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [32]:
print('Performance evaluation for pattern.en')

#ADAPT
accuracy_en_bert, unique_predicted_en_bert, confusion_matrix_en_bert, classification_report_en_bert = evaluate_performance(df_en_1_pos_neg, 'sentiment_bert', 'Label')

print('Accuracy: ', accuracy_en_bert,' Unique labels: ', unique_predicted_en_bert)

print('Confusion matrix: ')
print(confusion_matrix_en_bert)
print('Classification report: ')
print(classification_report_en_bert)

Performance evaluation for pattern.en
Accuracy for pattern.en: 0.00%
Unique labels for pattern.en:
[ 0.1         0.29444444  0.5         0.06666667  0.          0.05555556
 -0.6         0.31666667  0.07142857  0.4        -0.4         0.24833333]


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [33]:
print('Performance evaluation for nltk')

# Calculate the accuracy
accuracy_en_nltk = (df_en_1_pos_neg['sentiment_nltk'] == df_en_1_pos_neg['Label']).mean() * 100

# Print the accuracy
print("Accuracy for nltk: {:.2f}%".format(accuracy_en_nltk))



# Print unique predicted sentiment labels
print('Unique labels for nltk:')
print(df_en_1_pos_neg['sentiment_nltk'].unique())


# # Assign true and predicted labels
true_labels = df_en_1_pos_neg['Label']
predicted_labels = df_en_1_pos_neg['sentiment_nltk']

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Convert the confusion matrix to a DataFrame for better visualization
labels = np.unique(np.concatenate((true_labels, predicted_labels)))
cm_df_en_nltk = pd.DataFrame(cm, index=labels, columns=labels)

# Display the confusion matrix
print("Confusion Matrix for nltk:")
print(cm_df_en_nltk)



# Generate the classification report
report_en_nltk = classification_report(true_labels, predicted_labels)

# Display the classification report
print("Classification Report for nltk:")
print(report_en_nltk)

Performance evaluation for nltk
Accuracy for nltk: 0.00%
Unique labels for nltk:
[ 0.2263  0.5859  0.1531  0.8625  0.      0.3818 -0.4939  0.5106 -0.7096
  0.5975 -0.3612  0.8519]


ValueError: Classification metrics can't handle a mix of binary and continuous targets

## Spanish
TODO?

### Model 1: sentiment-analysis from spanish bert: beto-sentiment-analysis
TODO
https://huggingface.co/finiteautomata/beto-sentiment-analysis
Although this model was trained on tweets, not on news data, it is a popular model to use for sentiment analysis of spanish data.
The base model is BETO, a BERT model trained on spanish data (for further information: https://github.com/dccuchile/beto).

In [ ]:
sentiment_classifier_es_beto = pipeline('sentiment-analysis', model='finiteautomata/beto-sentiment-analysis')

In [ ]:
# Apply sentiment analysis on the 'data' column  and store the sentiment in a new column "sentiment_beto"
df_es_1['sentiment_beto'] = df_es_1['data'].apply(lambda x: sentiment_classifier_es_beto(x)[0]['score'])

# Print the updated dataframe
df_es_1.head()

### Model 2: sentiment-analysis from another spanish bert: bert-base-spanish-wwm-uncased
TODO

In [ ]:
sentiment_classifier_es_bert = pipeline('sentiment-analysis', model='dccuchile/bert-base-spanish-wwm-uncased')

In [ ]:
# Apply sentiment analysis on the 'data' column  and store the sentiment in a new column "sentiment_bert"
df_es_1['sentiment_bert'] = df_es_1['data'].apply(lambda x: sentiment_classifier_es_bert(x)[0]['score'])

# Print the updated dataframe
df_es_1.head()

#### Evaluate model performance for all spanish models
TODO 

In [ ]:
# Drop rows where 'Label' is NaN or empty
df_es_1.dropna(subset=['Label'], inplace=True)

# Print the updated DataFrame
df_es_1_pos_neg = df_es_1[df_es_1['Label'] != 'neutral']

In [ ]:
print('Performance evaluation for beto-sentiment-analysis')

# Calculate the accuracy
accuracy_es_beto = (df_es_1_pos_neg['sentiment_beto'] == df_es_1_pos_neg['Label']).mean() * 100

# Print the accuracy
print("Accuracy for beto: {:.2f}%".format(accuracy_es_beto))



# Print unique predicted sentiment labels
print('Unique labels for beto:')
print(df_es_1_pos_neg['sentiment_beto'].unique())


# # Assign true and predicted labels
true_labels = df_es_1_pos_neg['Label']
predicted_labels = df_es_1_pos_neg['sentiment_beto']

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Convert the confusion matrix to a DataFrame for better visualization
labels = np.unique(np.concatenate((true_labels, predicted_labels)))
cm_df_es_beto = pd.DataFrame(cm, index=labels, columns=labels)

# Display the confusion matrix
print("Confusion Matrix for beto:")
print(cm_df_es_beto)



# Generate the classification report
report_es_beto = classification_report(true_labels, predicted_labels)

# Display the classification report
print("Classification Report for beto:")
print(report_es_beto)

In [ ]:
print('Performance evaluation for bert-base-spanish-wwm-uncased')


# Calculate the accuracy
accuracy_es_bert = (df_es_1_pos_neg['sentiment_bert'] == df_es_1_pos_neg['Label']).mean() * 100

# Print the accuracy
print("Accuracy for bert: {:.2f}%".format(accuracy_es_bert))



# Print unique predicted sentiment labels
print('Unique labels for bert:')
print(df_es_1_pos_neg['sentiment_bert'].unique())


# # Assign true and predicted labels
true_labels = df_es_1_pos_neg['Label']
predicted_labels = df_es_1_pos_neg['sentiment_bert']

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Convert the confusion matrix to a DataFrame for better visualization
labels = np.unique(np.concatenate((true_labels, predicted_labels)))
cm_df_es_bert = pd.DataFrame(cm, index=labels, columns=labels)

# Display the confusion matrix
print("Confusion Matrix for bert:")
print(cm_df_es_bert)



# Generate the classification report
report_es_bert = classification_report(true_labels, predicted_labels)

# Display the classification report
print("Classification Report for beto:")
print(report_es_bert)

## English: data condensed
TODO

### Model 1: sentiment-analysis from bert-base-uncased
TODO
https://huggingface.co/bert-base-uncased

In [ ]:
# Model loaded above: sentiment_classifier_en = pipeline('sentiment-analysis', model='bert-base-uncased')

# Apply sentiment analysis on the 'data' column  and store the sentiment in a new column "sentiment_bert"
df_en_con['sentiment_bert'] = df_en_con['data'].apply(lambda x: sentiment_classifier_en(x)[0]['score'])

# Print the updated dataframe
df_en_con.head()

#### Evaluate model performance for english condensed model

In [ ]:
# Drop rows where 'Label' is NaN or empty
df_en_con.dropna(subset=['Label'], inplace=True)

# Print the updated DataFrame
df_en_con_pos_neg = df_en_con[df_en_1['Label'] != 'neutral']

In [ ]:
print('Performance evaluation for bert-base-uncased on english condensed')

# Calculate the accuracy
accuracy_en_con_bert = (df_en_con_pos_neg['sentiment_bert'] == df_en_con_pos_neg['Label']).mean() * 100

# Print the accuracy
print("Accuracy for bert: {:.2f}%".format(accuracy_en_con_bert))



# Print unique predicted sentiment labels
print('Unique labels for bert:')
print(df_en_con_pos_neg['sentiment_bert'].unique())


# # Assign true and predicted labels
true_labels = df_en_con_pos_neg['Label']
predicted_labels = df_en_con_pos_neg['sentiment_bert']

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Convert the confusion matrix to a DataFrame for better visualization
labels = np.unique(np.concatenate((true_labels, predicted_labels)))
cm_df_en_con_bert = pd.DataFrame(cm, index=labels, columns=labels)

# Display the confusion matrix
print("Confusion Matrix for bert:")
print(cm_df_en_con_bert)



# Generate the classification report
report_en_con_bert = classification_report(true_labels, predicted_labels)

# Display the classification report
print("Classification Report for bert:")
print(report_en_con_bert)